# Pipeline

This pipeline includes all steps in a single notebook (For Google Colab setup and testing purposes). It is currently a work-in-progress, for now, please visit ./notebooks for the individual steps.

In [47]:
# If you are running this pipeline on Google Colab,
# please update ENVIRONMENT to "GOOGLE_COLAB".
# Otherwise, please update ENVIRONMENT to "LOCAL".

ENVIRONMENT = "LOCAL"

In [48]:
# Recommended to set to True if the data has changed,
# otherwise set to "False" to save time
BYPASS_CACHE = False

# Recommended to set to True if you want to refresh raw dataset
# from the source (Zip file), otherwise set to False to save time
FORCE_UNZIP = False

# Set the random seed for reproducibility
RANDOM_SEED = 42

# Set the test and evaluation sizes
TEST_SIZE = 0.2
EVAL_SIZE = 0.1

# Calculate the train size
TRAIN_SIZE = 1 - TEST_SIZE - EVAL_SIZE

In [49]:
# Set global directories
DEFAULT_DIR = "/content" if ENVIRONMENT == "GOOGLE_COLAB" else "."

DATA_DIR = f"{DEFAULT_DIR}/data"
UTILS_DIR = f"{DEFAULT_DIR}/utils"
DATASET_DIR = f"{DATA_DIR}/datasets"
ZIPPED_RESOURCES_DIR = f"{DATA_DIR}/zipped"

# Caching
CACHE_DIR = f"{DATA_DIR}/cache"
DATASET_CACHE_DIR = f"{CACHE_DIR}/datasets"

# Datasets
TRAIN_DATA_DIR = f"{DATASET_DIR}/raw"
CLEANED_TRAIN_DATA_DIR = f"{DATASET_DIR}/cleaned"
PROCESSED_TRAIN_DATA_DIR = f"{DATASET_DIR}/processed"

TRAIN_DATA_CSV = f"{DATASET_DIR}/raw.csv"
CLEANED_TRAIN_DATA_CSV = f"{DATASET_DIR}/cleaned.csv"
PROCESSED_TRAIN_DATA_CSV = f"{DATASET_DIR}/processed.csv"

# Set path to cache visualizations and models
VISUALIZATION_DIR = f"{CACHE_DIR}/visualizations"
MODEL_DIR = f"{CACHE_DIR}/models"
MACOS_DIR = f"{DATASET_DIR}/__MACOSX"

In [50]:
# Path to raw dataset
GOOGLE_DRIVE_DIR = f"{DEFAULT_DIR}/drive"
GOOGLE_DRIVE_ROOT_DIR = f"{GOOGLE_DRIVE_DIR}/MyDrive"

ZIP_FILE = "Furniture_Data.zip"

DATASET_ZIP = (
    f"{GOOGLE_DRIVE_ROOT_DIR}/{ZIP_FILE}"
    if ENVIRONMENT == "GOOGLE_COLAB"
    else f"{DEFAULT_DIR}/{ZIP_FILE}"
)

DATASET_EXTRACT_DIR = (
    f"{DEFAULT_DIR}/Furniture_Data" if ENVIRONMENT == "GOOGLE_COLAB" else DATASET_DIR
)

In [51]:
if ENVIRONMENT == 'LOCAL':
    %pip install -r requirements.txt
elif ENVIRONMENT == "GOOGLE_COLAB":
    %pip install imagehash

Note: you may need to restart the kernel to use updated packages.


In [52]:
if ENVIRONMENT == "LOCAL":
    from utils.cache import zip_util_libs

    zip_util_libs(UTILS_DIR, f"{ZIPPED_RESOURCES_DIR}/utils.zip")

if ENVIRONMENT ==

In [54]:
from utils.cache import reload_custom_libraries

reload_custom_libraries()

from utils.converter import convert_to_df


train_df = convert_to_df(TRAIN_DATA_DIR, TRAIN_DATA_CSV)
cleaned_train_df = convert_to_df(CLEANED_TRAIN_DATA_DIR, CLEANED_TRAIN_DATA_CSV)
processed_train_df = convert_to_df(PROCESSED_TRAIN_DATA_DIR, CLEANED_TRAIN_DATA_CSV)

CSV file already exists at ./data/datasets/raw.csv. Loading from cache...
CSV file already exists at ./data/datasets/cleaned.csv. Loading from cache...
Directory ./data/datasets/processed does not exist. Skipping...


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print(
    "GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE"
)

In [ ]:
if ENVIRONMENT == "GOOGLE_COLAB":
    from google.colab import drive

    # Mount Google Drive
    drive.mount(GOOGLE_DRIVE_DIR, force_remount=True)

In [ ]:
from tqdm import tqdm
import zipfile
import shutil
import os

# if force unzip or TRAIN_DATA_DIR doesn't exist or empty
if FORCE_UNZIP or not tf.io.gfile.exists(TRAIN_DATA_DIR):
    if not os.path.isdir(f"{DATASET_EXTRACT_DIR}/Furniture_Data"):
        print("Starting unzip process...")
        # Extract the zip file
        with zipfile.ZipFile(DATASET_ZIP, "r") as zip_ref:
            zip_ref.extractall(DATASET_EXTRACT_DIR)
        print("Unzip process completed.")

    # Check if the new dataset already directory exists
    print("Checking for existing train dataset directory...")
    if os.path.isdir(TRAIN_DATA_DIR):
        # Delete the new dataset directory
        shutil.rmtree(TRAIN_DATA_DIR)
        print("Existing train dataset directory found and deleted.")

    print("Checking for `__MACOSX` directory...")
    # Check if the `__MACOSX` directory exists
    if os.path.isdir(MACOS_DIR):
        # Delete the `__MACOSX` directory
        shutil.rmtree(MACOS_DIR)
        print("`__MACOSX` directory found and deleted.")
    else:
        print("`__MACOSX` directory not found. Skipping deletion.")

    # Move the extracted dataset to the correct directory
    if os.path.isdir(f"{DATASET_EXTRACT_DIR}/Furniture_Data"):
        print("Moving extracted dataset to the correct directory...")
        shutil.move(f"{DATASET_EXTRACT_DIR}/Furniture_Data", TRAIN_DATA_DIR)
        print("Dataset moved successfully.")
else:
    print("Skipping unzip process as TRAIN_DATA_DIR exists and FORCE_UNZIP is not set.")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from IPython.display import display
from PIL import Image
from PIL import ImageFile
import os, sys
from tqdm.auto import tqdm

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(TRAIN_DATA_DIR)

In [ ]:
import os
import pandas as pd
import imagehash
from PIL import Image
from collections import defaultdict, Counter
from tqdm.auto import tqdm
import tensorflow as tf
import shutil


def image_category(directory):
    cat_list = pd.DataFrame(os.listdir(directory), columns=["Category"])
    for i in cat_list["Category"].values:
        if i == ".DS_Store" or i == "README.txt":
            cat_list.drop(cat_list[cat_list["Category"] == i].index, inplace=True)
    cat_list.reset_index(drop=True, inplace=True)
    return cat_list["Category"].values


def image_style(directory):
    style_list = [
        i
        for i in os.listdir(f"{directory}/beds")
        if os.path.isdir(os.path.join(directory, "beds", i))
        and i not in [".DS_Store", "README.txt"]
    ]
    return style_list


def image_path(directory, category):
    style = image_style(directory)
    image_files = []
    with tqdm(
        total=len(style), desc=f"Getting image path for category '{category}'"
    ) as pbar:
        for st in style:
            path = f"{directory}/{category}/{st}"
            for file in os.listdir(path):
                image_files.append(f"{path}/{file}")
            pbar.update(1)
    return image_files


def compute_hash(path):
    with Image.open(path) as img:
        hash_value = imagehash.average_hash(img)
    return hash_value


def imgResize(lists, size):
    with tqdm(total=len(lists), desc="Resizing images") as pbar:
        for item in lists:
            with Image.open(item) as img:
                img1 = img.resize(size, resample=0)
                img1.save(item, "JPEG")
            pbar.update(1)


def image_duplicate(category, image_files, style):
    hashes = {}
    unique_images = []
    style_images = [img for img in image_files if style in img]

    for image_file in tqdm(
        style_images,
        total=len(style_images),
        desc=f"Finding duplicate images in category '{category}', style '{style}'",
    ):
        try:
            image_hash = compute_hash(image_file)
        except PermissionError:
            print(f"Permission denied for file: {image_file}")
            continue

        if image_hash not in hashes:
            hashes[image_hash] = image_file
            unique_images.append(image_file)
    print(
        f"Category '{category}', Style '{style}': Found {len(style_images) - len(unique_images)} duplicate images."
    )
    return unique_images


def process_images(lists, category):
    category_dir = os.path.join(TRAIN_DATA_DIR, category)

    # Get the list of styles for the current category
    styles = [
        name
        for name in os.listdir(category_dir)
        if os.path.isdir(os.path.join(category_dir, name))
    ]

    # Delete images that does not belong to any style
    for img in lists:
        if not any(style in img for style in styles):
            os.remove(img)
            print(f"Deleted {img} due to unknown style.")

    # Get image sizes
    imageSize = []
    size_counter = Counter()
    with tqdm(total=len(lists), desc="Getting image size") as pbar:
        for item in lists:
            try:
                with Image.open(item) as img:
                    size = img.size
                    imageSize.append(size)
                    size_counter[size] += 1
            except PermissionError:
                print(f"Permission denied for file: {item}")
                continue
            pbar.update(1)
    print("224x224 pixels: ", size_counter[(224, 224)])
    print("350x350 pixels: ", size_counter[(350, 350)])
    print(
        "Other size: ", len(lists) - size_counter[(224, 224)] - size_counter[(350, 350)]
    )

    # Iterate over the styles
    for style in styles:
        cleaned_dir = os.path.join(CLEANED_TRAIN_DATA_DIR, category, style)

        # Check if cleaned images have already been saved
        if os.path.exists(cleaned_dir) and os.listdir(cleaned_dir):
            print(
                f"Category '{category}', Style '{style}': Cleaned images are already available."
            )
            continue

        # Get unique images for the current style
        unique_images = image_duplicate(category, lists, style)

        # Save unique images to CLEANED_TRAIN_DATA_DIR
        os.makedirs(cleaned_dir, exist_ok=True)
        with tqdm(
            total=len(unique_images),
            desc=f"Saving unique images for category '{category}', style '{style}'",
        ) as pbar:
            for img_path in unique_images:
                try:
                    shutil.copy(img_path, cleaned_dir)
                except PermissionError:
                    print(f"Permission denied for file: {img_path}")
                    continue
                pbar.update(1)

        print(
            f"Category '{category}', Style '{style}': Cleaned images saved successfully."
        )

    print(f"Category '{category}': All styles processed successfully.")
    return imageSize

In [ ]:
# Define the categories
categories = ["beds", "chairs", "dressers", "lamps", "sofas", "tables"]

# Extract paths for each category
paths = {category: image_path(TRAIN_DATA_DIR, category) for category in categories}

print("Successfully extracted all paths!")

In [ ]:
# Calculate the total
total = 0

# Print the amount for each category
for category in categories:
    amount = len(paths[category])
    print(f"Amount of {category}: {amount}")
    total += amount

# Print the total
print("Total:", total)

In [ ]:
plt.bar(paths.keys(), [len(v) for v in paths.values()], width=0.5)
plt.title("Number of images in each category")
plt.xlabel("Categories")
plt.ylabel("Amount")
plt.show()

## Get duplicate images from each categories
### Check duplicants and remove duplicants from each categories

In [ ]:
# Create a new dictionary to store the paths and image sizes for each category
data = {}
for category, items in paths.items():
    imageSizes = process_images(items, category)
    data[category] = {
        "paths": image_path(CLEANED_TRAIN_DATA_DIR, category),
        "imageSizes": imageSizes,
    }

print("Successfully extracted all paths and image sizes!")

### Recheck the categories after removing duplicants

In [ ]:
# Calculate the total
total = 0

# Print the amount for each category
for category in categories:
    amount = len(data[category]["paths"])
    print(f"Amount of {category}: {amount}")
    total += amount

# Print the total
print("Total:", total)

### Visualization

In [ ]:
plt.bar(
    categories, [len(data[category]["paths"]) for category in categories], width=0.5
)
plt.title("Number of images in each category")
plt.xlabel("Categories")
plt.ylabel("Amount")

-   In the plot we can see the differences between of the amount in different columns. Because of that, we will need to check for the imbalances of our dataset

## Get different amount of image size of each category

In [ ]:
# Define a dictionary to store the dataframes
# each category in df_dims is data[category][imageSizes]
df_dims = {
    category: pd.DataFrame(data[category]["imageSizes"], columns=["width", "height"])
    for category in categories
}

print("Successfully extracted all dimensions!")

In [ ]:
import matplotlib.pyplot as plt

# Define the number of rows and columns for the subplot grid
n_rows = 2
n_cols = 3

# Create the subplots
fig, axs = plt.subplots(n_rows, n_cols, figsize=(15, 10))

# Flatten the axes array
axs = axs.flatten()

# Iterate over the categories and axes
for ax, category in zip(axs, categories):
    # Create the scatter plot for the current category
    df_dims[category].plot.scatter(
        x="width", y="height", ax=ax, title=category.capitalize()
    )

# Adjust the layout
plt.tight_layout()
plt.show()

In [ ]:
# Iterate over the categories
for category in categories:
    print(category.capitalize())
    display(df_dims[category].describe().T)

- The dataset contains 6 categories of items with 90083 items total
- The dataset contains all file with JPEG type
- The dataset contains image mainly in two dimensions: 224x224 and 350x350. However there are one anomaly dimensions in the table category which is: 500x446